# Rotonda Garza Sada

In [8]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import IPython
import random
import json


In [9]:

# Lane decisions
# Por donde entra
lane = ['a', 'b', 'c', 'd']
lane2 = [0, 1, 2, 3]

# Trajectories
# A -> DERECHA
inA = [(12, 6), (11, 6), (10, 6)]
outA = [(10, 4), (11, 4), (12, 4)]

# B -> NORTE
inB = [(6, 9), (6, 8)]
outB = [(7, 8), (7, 9)]

# C -> IZQUIERDA
inC = [(2, 4), (3, 4), (4, 4)]
outC = [(4, 6), (3, 6), (2, 6)]

# D -> SUR
inD = [(8, 2)]
outD = [(7, 1)]

# COORD -> ROTONDA
roundabout = [(9, 4), (9, 6), (7, 7), (6, 7), (5, 6), (5, 4), (7, 3), (8, 3)]

# COORD -> SEMAFORO
traffic_li = [(10, 6), (6, 8), (4, 4), (8, 2)]


# Definición del Agente

In [10]:
# Agente Automovil

class carAgent(ap.Agent):

    # Setup
    def setup(self):
        # Constructor
        # Puntos coordenados
        self.x = 0
        self.y = 0
        self.isCar = True
        # Ya ha spawneado, ya en rotonda, saliendo rotonda
        self.spawned = False
        self.inRoundabout = False
        self.exiting = False
        # ????
        self.rIndex = 0
        self.lIndex = 0
        # IDEA BRANDN
        self.inicioBX = 0
        self.inicioBY = 0
        self.finalBX = 0
        self.finalBY = 0
        self.iteradorr = 0

        # Inicio de agente
        self.ruta = []
        self.rutaReal = []

        # 1 -> A, 2 -> B, 3 -> C, 4 -> D
        

        # OBTENCIÓN DEL SOURCE
        self.source = random.choice(lane2)
        self.sourceA = random.choice(lane)
        self.inicio()

        self.destination = self.source
        self.destinationA = self.sourceA

       # WHILE PARA EVITAR SALIDA POR MISMA ENTRADA
        while self.source == self.destination:
            self.destination = random.choice(lane2)

        # While para lo de alfonso
        while self.sourceA == self.destinationA:
            self.destinationA = random.choice(lane)

        # INICIALIZA LA POSICIÓN
        self.principio()
        # iguala la posicion
        self.next_step = self.ruta[0]

        # FUNCIÓN PARA ENCONTRAR LA SALIDA CORRECTA
        if self.source < self.destination:
            inicio = 2*(self.source+1)-1
            final = inicio + 1 + 2*(self.destination-(self.source+1))
        else:
            b = self.destination + 4
            inicio = 2*(self.source+1)-1
            final = inicio + 1 + 2*(b-(self.source+1))

        self.inicioInicial = roundabout[inicio]

        # BUSCANDO EL SLICE DE ROUNDABOUT QUE CUMPLA LA RUTA
        self.rotonda(inicio, final)
        # APENDIZANDO EL SLICE FINAL DE LA SALIDA
        self.final()

    # DEFINE EL INICIO DE LA RUTA -> BRANDON
    def inicio(self):
        if self.source == 0:
            self.ruta.extend(inA[:])
        elif self.source == 1:
            self.ruta.extend(inB[:])
        elif self.source == 2:
            self.ruta.extend(inC[:])
        else:
            self.ruta.extend(inD[:])

    # DEFINE LA RUTA DE LA ROTONDA -> BRANDON
    def rotonda(self, inicio, fin):
        inter = fin - inicio + 1
        for i in range(inter):
            if (inicio == 8):
                inicio = 0
                self.ruta.append(roundabout[inicio])
            else:
                self.ruta.append(roundabout[inicio])
            inicio += 1

    # DEFINE EL FINAL DE LA RUTA DE LA ROTONDA -> BRANDON
    def final(self):
        if self.destination == 0:
            self.ruta.extend(outA[:])
        elif self.destination == 1:
            self.ruta.extend(outB[:])
        elif self.destination == 2:
            self.ruta.extend(outC[:])
        else:
            self.ruta.extend(outD[:])

    # Inicio Alfonso
    def principio(self):

        if self.sourceA == 'a':
            self.x, self.y = 12, 6
        elif self.sourceA == 'b':
            self.x, self.y = 6, 9
        elif self.sourceA == 'b':
            self.x, self.y = 2, 4
        else:
            self.x, self.y = 8, 2

        self.next_step = tuple([self.x, self.y])

    # FUNCION PARA MOVERSE
    def move(self):

        if (self.iteradorr < len(self.ruta)):
            self.x = self.ruta[self.iteradorr][0]
            self.y = self.ruta[self.iteradorr][1]

        self.iteradorr += 1

        # if(self.iteradorr != len(self.ruta)-1):
        #   self.iteradorr += 1
        #  self.x = self.ruta[self.iteradorr][0]
        # self.y = self.ruta[self.iteradorr][1]
        #print(f'Self X: {self.x} , {self.y}')
        # if(self.iteradorr == len(self.ruta)):
        #   self.x = self.ruta[len(self.ruta)][0]
        #  self.y = self.ruta[len(self.ruta)][1]
        # print(f'Self X: {self.x} , {self.y}')

        #self.iteradorr += 1
        #self.next_step = self.ruta[self.iteradorr]

    # CALCULAR LA SIGUIENTE COORDENADA
    def calculate_next(self):
        self.move()
        if (self.iteradorr < len(self.ruta)):
            self.next_step = self.ruta[self.iteradorr]
       # self.next_step = self.ruta[self.iteradorr]
        # print(self.iteradorr)
        #print(f'Car {car} tiene coordenadas ({self.next_step})')
        #print(f'Car {car} tiene ruta ({self.ruta})')


# Agente Semaforo

In [11]:
class semaforoAgent(ap.Agent):

    def setup(self):

        self.x = 0
        self.y = 0
        self.isGreen = False
        self.waitTime = 0
        self.isCar = False
        self.spawned = True
        

# Definicion del Modelo

In [12]:
# MODELO DE LA ROTONDA
class rotondaModel(ap.Model):

    # CONSTRUCTOR DEL MODELO
    def setup(self):
        self.iterador = 0
        self.c = 0
        self.jsonfinal = []
        cars = self.agents = ap.AgentList(self, self.p.agents, carAgent)

        # Create grid (rotonda)
        self.rotonda = ap.Grid(self, [self.p.size]*2, track_empty=True)
        self.spawning_cars = []
        # I HASTA MI NUMERO DE PARÁMETROS EN EL AGENTE
        for i in range(self.p.agents):
            # INICIO EN EL JAIL 0,0 FUERA DE LA SIMULCIÓN
            self.my_tuple = tuple([0, 0])
            self.spawning_cars.append(self.my_tuple)
        # INICIALIZANDO LOS AGENTES EN LA SIMULACIÓN EN (0,0)
        self.rotonda.add_agents(cars, self.spawning_cars)

        # Iniciar semáforos
        # A = -1, B = -2, C = -3, D = -4
        #semaforos = ap.AgentList(self, 4, semaforoAgent)
        #self.rotonda.add_agents(semaforos, [tuple([-1, -1]), tuple([-2, -2]), tuple([-3, 3]), tuple([-4, -4])])
        # A = 0, B = 1, C = 2, D = 3
        
        # Inicializar semaforos y sus tiempos de espera
        self.semaforoA = []
        self.semaforoB = []
        self.semaforoC = []
        self.semaforoD = []
        self.semaforos = [False, False, False, False]
        self.semaforosEspera = [0,0,0,0]
        
        # Seleccionar un semáforo a encender
        self.semaforoActivo = random.choice(lane2)
        self.semaforos[self.semaforoActivo] = True

    def calcularPesoSemaforo(self):
        self.semaforosWeight = []
        self.carCount = 0
        cars = self.agents
        for i in range(4):
            if i == 0:
                for car in cars:
                    if tuple([car.x,car.y]) in inA:
                        self.carCount = self.carCount + 1
            if i == 1:
                for car in cars:
                    if tuple([car.x,car.y]) in inB:
                        self.carCount = self.carCount + 1
            if i == 2:
                for car in cars:
                    if tuple([car.x,car.y]) in inC:
                        self.carCount = self.carCount + 1
            if i == 3:
                for car in cars:
                    if tuple([car.x,car.y]) in inD:
                        self.carCount = self.carCount + 1
            result = self.semaforosEspera[i] * self.carCount
            self.carCount = 0
            self.semaforosWeight.append(result)

    def step(self):
        bandera = 0
        self.c += 1
        # SPAWNEO MI POSICIÓN 1 DE AGENTE
        self.agents[self.iterador].spawned = True
        # SI MI ITERADOR ES MENOR A NUM AGENTS - 1
        if self.iterador < (self.p.agents - 1):
            # LE SUMO UNO, YA SPAWNEE TODOS LOS 18 PARA QUE DIGA 20
            self.iterador = self.iterador + 1


        # CAMBIAR ESTADO DE SEMAFOROS
        # 5 es el numero minimo de steps a esperar antes de cambiar
        if self.semaforosEspera[self.semaforoActivo] < 5:
            for i in range(4):
                self.semaforosEspera[i] = self.semaforosEspera[i]+1
        else:
            self.calcularPesoSemaforo()
            self.maxWeight = 0
            self.maxSemaforo = 0
            for i in range(4):
                if self.semaforosWeight[i] > self.maxWeight:
                    self.maxWeight = self.semaforosWeight[i]
                    self.maxSemaforo = i
                    self.semaforoActivo = i
            
            for i in range(4):
                if i == self.maxSemaforo:
                    self.semaforos[i] = True
                    self.semaforosEspera[i] = 0
                else :
                    self.semaforos[i] = False
                    self.semaforosEspera[i] = self.semaforosEspera[i]+1

        #print(self.semaforos)
        self.semaforoA.append(self.semaforos[0])
        self.semaforoB.append(self.semaforos[1])
        self.semaforoC.append(self.semaforos[2])
        self.semaforoD.append(self.semaforos[3])
        # SIMULAR AGENTES QUE YA ESTÁN EN TRUE
        cars = self.agents.select(self.agents.spawned == True)
        # POR CADA CARO EN EL ARREGLO DE CARS
        self.cont = 0
        self.jsonfinal = cars
        for car in cars:
            #ITERAR CUADRITOS VACIOS HASTA ENCONTRAR EL NEXT STEP
            # for square in self.rotonda.empty:
            for car2 in cars:
                
                if (car != car2 and (car.next_step[0] == car2.x and car.next_step[1] == car2.y) and (car.next_step not in [(7, 1), (2, 6), (7, 9), (12, 4)])):
                    bandera = 1
                
                if (car.next_step == car.inicioInicial and self.semaforos[0] == False and car.source == 0):
                    bandera = 1
                if (car.next_step == car.inicioInicial and self.semaforos[1] == False and car.source == 1):
                    bandera = 1
                if (car.next_step == car.inicioInicial and self.semaforos[2] == False and car.source == 2):
                    bandera = 1
                if (car.next_step == car.inicioInicial and self.semaforos[3] == False and car.source == 3):
                    bandera = 1
                

            if bandera == 0:
                self.rotonda.move_to(car, car.next_step)
                car.calculate_next()

            bandera = 0
            # MUEVE EL AGENTE A LA SIG. COORDENADA
            # move_to = agent.py grid

            # SIGUIENTE COORDENADA
            # ITERAR CUADRITOS VACIOS HASTA ENCONTRAR EL NEXT STEP
            # for square in self.rotonda.empty:
            '''
            for car2 in cars:
                if (car.isCar == True and car2.isCar):
                    if (car != car2 and (car.next_step[0] == car2.x and car.next_step[1] == car2.y) and (car.next_step not in [(7, 1), (2, 6), (7, 9), (12, 4)])):
                        bandera = 1

                    if bandera == 0:
                        self.rotonda.move_to(car, car.next_step)
                        car.calculate_next()
                    bandera = 0
            # MUEVE EL AGENTE A LA SIG. COORDENADA
            # move_to = agent.py grid
            '''
            # SIGUIENTE COORDENADA
        
        todosCarros = self.agents
        #.select(self.agents.isCar == True)
        for car in todosCarros:
            if car.spawned == False:
                car.rutaReal.append(tuple([0, 0]))
            else: 
                car.rutaReal.append(tuple([car.x, car.y]))

        
        t = 0
        my_dict = {}
        if(self.c == self.p.steps):
            for car in cars:
                my_dict[t] = car.rutaReal
                #print(f'Carro: {t}, ruta: {car.ruta}')
                t += 1
            
            #print(my_dict)
            #json_obj = json.dumps(my_dict)
            # print(json_obj)
            with open('./data.json', 'w') as file:
                json.dump(my_dict, file, indent=4)

            my_dict = {}
            my_dict[0] = self.semaforoA
            my_dict[1] = self.semaforoB
            my_dict[2] = self.semaforoC
            my_dict[3] = self.semaforoD
            #json_obj = json.dumps(my_dict)
            # print(json_obj)
            with open('./dataSems.json', 'w') as file:
                json.dump(my_dict, file, indent=4)
    
    


# Animacion

In [13]:
parameters = {

    'size': 13,  # Height and length of the grid
    'steps': 100,
    'agents': 8,  # Percentage of grid covered by trees
}


In [14]:
def animation_plot(model, ax):
    attr_grid = model.rotonda.attr_grid('spawned')
    color_dict = {False: '#d5e5d5', True: '#d62c2c', None: '#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a roundabout\n"
                 f"Time-step: {model.t}")


fig, ax = plt.subplots()
model = rotondaModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=10))
